This tutorial shows you have you can download data from a public endpoint and transform it into a Snowpark Dataframe and save the results into a table in Snowflake.

**Note:** Running this notebook require that you have ACCOUNTADMIN or SECURITYADMIN roles to create new network rules.

In [ ]:
CREATE or replace TABLE bike_riders (
  timestamp STRING,
  northbound NUMBER,
  southbound NUMBER
);

By default, Snowflake restricts network traffic from requests from public IP addresses. In order to access data from externally, we first need to create an external access integration to add `data.seattle.gov` as an allowed endpoint.

In [ ]:
CREATE OR REPLACE NETWORK RULE SEATTLE_OPEN_DATA_RULE
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = ('DATA.SEATTLE.GOV');

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION SEATTLE_OPEN_DATA_INTEGRATION
ALLOWED_NETWORK_RULES = (SEATTLE_OPEN_DATA_RULE)
ENABLED=TRUE;

Next, we create an external function that allows users to connect outside of Snowflake and fetch the data from the remote endpoint. We attach the external access object that we created earlier to the external function so that it has permission to access the allowed network. The external function uses the `requests` library in Python to get the JSON response from the URL.

In [ ]:
CREATE OR REPLACE FUNCTION FETCH_ENDPOINT(URL STRING)
returns string
language python
runtime_version=3.8
handler = 'fetch_and_transform_data'
external_access_integrations=(SEATTLE_OPEN_DATA_INTEGRATION)
packages = ('requests')
as
$$
import requests
import _snowflake
session = requests.Session()
def fetch_and_transform_data(url):
    response = requests.get(url)
    data = response.json()
    # Perform data transformation here
    return data
$$;

Now we can call the external function on [this URL](https://data.seattle.gov/resource/65db-xm6k.json), we see that the data is a JSON list, so we use Snowflake's [`PARSE_JSON`](https://docs.snowflake.com/en/sql-reference/functions/parse_json) function to process the data. 

In [ ]:
select fetch_endpoint('https://data.seattle.gov/resource/65db-xm6k.json')

In [ ]:
select PARSE_JSON(fetch_endpoint('https://data.seattle.gov/resource/65db-xm6k.json'))

Now we wanted to get the demo output of the query. What we learn is that we have an extraneous column of "total", so we'll get rid of that before inserting the data

In [ ]:
insert into bike_riders
with json_blob as 
(select parse_json(fetch_endpoint('https://data.seattle.gov/resource/65db-xm6k.json')) AS json_arr)
select 
   value:date::STRING AS date,
   value:fremont_bridge_nb::NUMBER AS northbound,
   value:fremont_bridge_sb::NUMBER AS southbound
from json_blob, TABLE(FLATTEN(input => json_arr))

In [ ]:
select * from bike_riders

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
df = session.table("bike_riders")
df

Now you can use this Snowpark Dataframe to work with your data in Python!

In [ ]:
df.describe()